In [1]:
!pip install salesforce-lavis

In [2]:
import os
import json

import pandas as pd
from tqdm import tqdm
from lavis.common.utils import get_abs_path, get_cache_path

/usr/local/lib/python3.10/dist-packages/fairscale/experimental/nn/offload.py:19: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  return torch.cuda.amp.custom_fwd(orig_func)  # type: ignore
/usr/local/lib/python3.10/dist-packages/fairscale/experimental/nn/offload.py:30: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  return torch.cuda.amp.custom_bwd(orig_func)  # type: ignore


In [5]:
# prompt: buat annotasi pada file /content/Coconut_cream_0014.jpg yang memuat tingkat kecerahan, nama file,tingkat kepekatan warna, bahan utama, cara penyajiannya menggunakan lavis yang sudah saya install

import os
import json
import pandas as pd
from tqdm import tqdm
from lavis.common.utils import get_abs_path, get_cache_path
from PIL import Image
from lavis.models import load_model_and_preprocess

# Load the model
model, vis_processors, _ = load_model_and_preprocess(name="blip_caption", model_type="base_coco", is_eval=True)

# Image path
image_path = "/content/Coconut_cream_0014.jpg"

# Load the image
raw_image = Image.open(image_path).convert("RGB")

# Preprocess the image
image = vis_processors["eval"](raw_image).unsqueeze(0).cuda()

# Generate caption
caption = model.generate({"image": image})

# Extract information (you may need to refine this part based on the caption quality)
# Example:
brightness_level = "unknown"  # placeholder
color_saturation = "unknown"  # placeholder
main_ingredient = "unknown" # placeholder
serving_method = "unknown" # placeholder
file_name = os.path.basename(image_path)

# You might need to use more sophisticated NLP to extract more information from the caption.
# For example:
# - Use regular expressions to find keywords
# - Use a named entity recognition (NER) model
# - Use a more advanced language model for better semantic understanding

print(f"Caption: {caption}")
print(f"Filename: {file_name}")
print(f"Brightness level: {brightness_level}")
print(f"Color saturation: {color_saturation}")
print(f"Main Ingredient: {main_ingredient}")
print(f"Serving Method: {serving_method}")

# Create a dictionary to store the results
annotation = {
    "filename": file_name,
    "brightness": brightness_level,
    "color_saturation": color_saturation,
    "main_ingredient": main_ingredient,
    "serving_method": serving_method,
    "caption": caption,
}

# Save the annotations in json format (optional)
# with open("image_annotations.json", "w") as f:
#     json.dump(annotation, f, indent=4)

# print("Annotations saved to image_annotations.json")

RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same

In [6]:
import os
import json
from PIL import Image
import torch
from lavis.models import load_model_and_preprocess

# Define device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the model
model, vis_processors, _ = load_model_and_preprocess(name="blip_caption", model_type="base_coco", is_eval=True)
model = model.to(device)  # Move model to the appropriate device

# Image path
image_path = "/content/Coconut_cream_0014.jpg"

# Load the image
raw_image = Image.open(image_path).convert("RGB")

# Preprocess the image
image = vis_processors["eval"](raw_image).unsqueeze(0).to(device)

# Generate caption
caption = model.generate({"image": image})[0]  # Get the generated caption

# File information
file_name = os.path.basename(image_path)

# Placeholder logic for brightness, color saturation, ingredients, serving method
# You can enhance this using NLP or image processing techniques
if "bright" in caption.lower():
    brightness_level = "bright"
elif "dark" in caption.lower():
    brightness_level = "dark"
else:
    brightness_level = "neutral"

if "colorful" in caption.lower() or "vivid" in caption.lower():
    color_saturation = "high"
elif "dull" in caption.lower():
    color_saturation = "low"
else:
    color_saturation = "medium"

if "coconut" in caption.lower():
    main_ingredient = "coconut"
else:
    main_ingredient = "unknown"

if "served" in caption.lower():
    serving_method = caption.split("served")[1].strip()
else:
    serving_method = "unknown"

# Create annotation dictionary
annotation = {
    "filename": file_name,
    "brightness": brightness_level,
    "color_saturation": color_saturation,
    "main_ingredient": main_ingredient,
    "serving_method": serving_method,
}

# Print annotation
print(json.dumps(annotation, indent=4))

# Save annotations to a JSON file (optional)
with open("image_annotations.json", "w") as f:
    json.dump(annotation, f, indent=4)

print("Annotations saved to image_annotations.json")


{
    "filename": "Coconut_cream_0014.jpg",
    "brightness": "neutral",
    "color_saturation": "medium",
    "main_ingredient": "unknown",
    "serving_method": "unknown"
}
Annotations saved to image_annotations.json


In [7]:
import os
import json
import cv2
import numpy as np
from PIL import Image
import torch
from lavis.models import load_model_and_preprocess
import re

# Define the device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load BLIP model
model, vis_processors, _ = load_model_and_preprocess(
    name="blip_caption", model_type="base_coco", is_eval=True
)
model = model.to(device)  # Move model to device

# Image path
image_path = "/content/Coconut_cream_0014.jpg"

# Step 1: Calculate brightness and saturation using OpenCV
def calculate_brightness_saturation(image_path):
    # Load image in BGR format
    img = cv2.imread(image_path)

    # Convert to HSV
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # Brightness: Mean of Value channel in HSV
    brightness = np.mean(hsv[:, :, 2]) / 255  # Normalize to [0, 1]

    # Saturation: Mean of Saturation channel in HSV
    saturation = np.mean(hsv[:, :, 1]) / 255  # Normalize to [0, 1]

    return round(brightness, 2), round(saturation, 2)

# Step 2: Generate caption using BLIP
def generate_caption(image_path, model, vis_processors, device):
    # Load and preprocess the image
    raw_image = Image.open(image_path).convert("RGB")
    image = vis_processors["eval"](raw_image).unsqueeze(0).to(device)

    # Generate caption
    caption = model.generate({"image": image})[0]
    return caption

# Step 3: Extract main ingredient and serving method from caption
def extract_details_from_caption(caption):
    # Extract serving method
    if "served" in caption:
        match = re.search(r"served (.*?)[.,]", caption)
        serving_method = match.group(1).strip() if match else "unknown"
    else:
        serving_method = "unknown"

    # Extract main ingredient
    if "coconut" in caption.lower():
        main_ingredient = "coconut"
    elif "cream" in caption.lower():
        main_ingredient = "cream"
    else:
        main_ingredient = "unknown"

    return main_ingredient, serving_method

# Run the pipeline
brightness, saturation = calculate_brightness_saturation(image_path)
caption = generate_caption(image_path, model, vis_processors, device)
main_ingredient, serving_method = extract_details_from_caption(caption)

# Get filename
file_name = os.path.basename(image_path)

# Create annotation dictionary
annotation = {
    "filename": file_name,
    "brightness": brightness,
    "color_saturation": saturation,
    "main_ingredient": main_ingredient,
    "serving_method": serving_method,
    "caption": caption,
}

# Print the annotation
print(json.dumps(annotation, indent=4))

# Save annotations to a JSON file (optional)
with open("image_annotations.json", "w") as f:
    json.dump(annotation, f, indent=4)

print("Annotations saved to image_annotations.json")


{
    "filename": "Coconut_cream_0014.jpg",
    "brightness": 0.74,
    "color_saturation": 0.39,
    "main_ingredient": "unknown",
    "serving_method": "unknown",
    "caption": "a pie sitting on top of a pan on a table"
}
Annotations saved to image_annotations.json


In [9]:
import os
import json
import cv2
import numpy as np
from PIL import Image
import torch
from lavis.models import load_model_and_preprocess
from transformers import pipeline

# Define the device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load BLIP model
model, vis_processors, _ = load_model_and_preprocess(
    name="blip_caption", model_type="base_coco", is_eval=True
)
model = model.to(device)  # Move model to device

# Load lightweight LLM pipeline using PyTorch
llm = pipeline("text2text-generation", model="google/flan-t5-large", framework="pt")

# Step 1: Calculate brightness and saturation using OpenCV
def calculate_brightness_saturation(image_path):
    # Load image in BGR format
    img = cv2.imread(image_path)

    # Convert to HSV
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # Brightness: Mean of Value channel in HSV
    brightness = np.mean(hsv[:, :, 2]) / 255  # Normalize to [0, 1]

    # Saturation: Mean of Saturation channel in HSV
    saturation = np.mean(hsv[:, :, 1]) / 255  # Normalize to [0, 1]

    return round(brightness, 2), round(saturation, 2)

# Step 2: Generate caption using BLIP
def generate_caption(image_path, model, vis_processors, device):
    # Load and preprocess the image
    raw_image = Image.open(image_path).convert("RGB")
    image = vis_processors["eval"](raw_image).unsqueeze(0).to(device)

    # Generate caption
    caption = model.generate({"image": image})[0]
    return caption

# Step 3: Extract main ingredient and serving method using LLM
def extract_details_with_llm(caption, llm):
    # Prompt to LLM for extracting details
    prompt = f"""
    Based on the caption: "{caption}", extract the following details:
    1. Main ingredient of the dish.
    2. Method of serving or presentation.
    Provide the answer in JSON format with keys 'main_ingredient' and 'serving_method'.
    """

    # Generate response
    response = llm(prompt, max_length=100)[0]["generated_text"]

    # Parse response into a dictionary
    try:
        details = json.loads(response)
    except json.JSONDecodeError:
        details = {"main_ingredient": "unknown", "serving_method": "unknown"}

    return details

# Main Pipeline
def annotate_image(image_path, model, vis_processors, llm, device):
    # Step 1: Calculate brightness and saturation
    brightness, saturation = calculate_brightness_saturation(image_path)

    # Step 2: Generate caption using BLIP
    caption = generate_caption(image_path, model, vis_processors, device)

    # Step 3: Extract details with LLM
    details = extract_details_with_llm(caption, llm)

    # Get filename
    file_name = os.path.basename(image_path)

    # Create annotation dictionary
    annotation = {
        "filename": file_name,
        "brightness": brightness,
        "color_saturation": saturation,
        "main_ingredient": details.get("main_ingredient", "unknown"),
        "serving_method": details.get("serving_method", "unknown"),
        "caption": caption,
    }

    return annotation

# Image path
image_path = "/content/Coconut_cream_0014.jpg"

# Run the pipeline
annotation = annotate_image(image_path, model, vis_processors, llm, device)

# Print the annotation
print(json.dumps(annotation, indent=4))

# Save annotations to a JSON file (optional)
with open("image_annotations.json", "w") as f:
    json.dump(annotation, f, indent=4)

print("Annotations saved to image_annotations.json")



model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

{
    "filename": "Coconut_cream_0014.jpg",
    "brightness": 0.74,
    "color_saturation": 0.39,
    "main_ingredient": "unknown",
    "serving_method": "unknown",
    "caption": "a pie sitting on top of a pan on a table"
}
Annotations saved to image_annotations.json


In [10]:
import os
import json
import cv2
import numpy as np
from PIL import Image
import torch
from lavis.models import load_model_and_preprocess
from transformers import pipeline

# Define the device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load BLIP model
model, vis_processors, _ = load_model_and_preprocess(
    name="blip_caption", model_type="base_coco", is_eval=True
)
model = model.to(device)  # Move model to device

# Load lightweight LLM pipeline using PyTorch
llm = pipeline("text2text-generation", model="google/flan-t5-large", framework="pt")

# Step 1: Calculate brightness and saturation using OpenCV
def calculate_brightness_saturation(image_path):
    # Load image in BGR format
    img = cv2.imread(image_path)

    # Convert to HSV
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # Brightness: Mean of Value channel in HSV
    brightness = np.mean(hsv[:, :, 2]) / 255  # Normalize to [0, 1]

    # Saturation: Mean of Saturation channel in HSV
    saturation = np.mean(hsv[:, :, 1]) / 255  # Normalize to [0, 1]

    return round(brightness, 2), round(saturation, 2)

# Step 2: Generate caption using BLIP
def generate_caption(image_path, model, vis_processors, device):
    # Load and preprocess the image
    raw_image = Image.open(image_path).convert("RGB")
    image = vis_processors["eval"](raw_image).unsqueeze(0).to(device)

    # Generate caption
    caption = model.generate({"image": image})[0]
    return caption

# Step 3: Extract ingredients and presentation using LLM
def extract_details_with_llm(caption, llm):
    # Prompt to LLM for extracting details
    prompt = f"""
    Based on the caption: "{caption}", extract the following details:
    1. List all ingredients mentioned in the caption.
    2. Describe the method of presentation (e.g., sliced, plated, etc.).
    Provide the answer in JSON format with keys 'ingredients' and 'presentation'.
    """

    # Generate response
    response = llm(prompt, max_length=200)[0]["generated_text"]

    # Parse response into a dictionary
    try:
        details = json.loads(response)
    except json.JSONDecodeError:
        details = {"ingredients": "unknown", "presentation": "unknown"}

    return details

# Main Pipeline
def annotate_image(image_path, model, vis_processors, llm, device):
    # Step 1: Calculate brightness and saturation
    brightness, saturation = calculate_brightness_saturation(image_path)

    # Step 2: Generate caption using BLIP
    caption = generate_caption(image_path, model, vis_processors, device)

    # Step 3: Extract details with LLM
    details = extract_details_with_llm(caption, llm)

    # Get filename
    file_name = os.path.basename(image_path)

    # Create annotation dictionary
    annotation = {
        "filename": file_name,
        "brightness": brightness,
        "color_saturation": saturation,
        "ingredients": details.get("ingredients", "unknown"),
        "presentation": details.get("presentation", "unknown"),
        "caption": caption,
    }

    return annotation

# Image path
image_path = "/content/Pizza_pugliese_0088.jpg"

# Run the pipeline
annotation = annotate_image(image_path, model, vis_processors, llm, device)

# Print the annotation
print(json.dumps(annotation, indent=4))

# Save annotations to a JSON file (optional)
with open("image_annotations.json", "w") as f:
    json.dump(annotation, f, indent=4)

print("Annotations saved to image_annotations.json")


{
    "filename": "Pizza_pugliese_0088.jpg",
    "brightness": 0.82,
    "color_saturation": 0.32,
    "ingredients": "unknown",
    "presentation": "unknown",
    "caption": "a white plate topped with a pizza covered in toppings"
}
Annotations saved to image_annotations.json


In [12]:
import os
import json
import cv2
import numpy as np
from PIL import Image
import torch
from transformers import pipeline

# Define the device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load lightweight LLM pipeline using PyTorch
llm = pipeline("text2text-generation", model="google/flan-t5-large", framework="pt")

# Step 1: Calculate brightness and saturation using OpenCV
def calculate_brightness_saturation(image_path):
    # Load image in BGR format
    img = cv2.imread(image_path)

    # Convert to HSV
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # Brightness: Mean of Value channel in HSV
    brightness = np.mean(hsv[:, :, 2]) / 255  # Normalize to [0, 1]

    # Saturation: Mean of Saturation channel in HSV
    saturation = np.mean(hsv[:, :, 1]) / 255  # Normalize to [0, 1]

    return round(brightness, 2), round(saturation, 2)

# Step 2: Read caption from a text file
def read_caption_from_file(image_path):
    # Extract file name without extension
    base_name = os.path.splitext(os.path.basename(image_path))[0]

    # Construct the path for the text file
    caption_file_path = os.path.join(os.path.dirname(image_path), f"{base_name}.txt")

    # Check if the caption file exists
    if not os.path.exists(caption_file_path):
        raise FileNotFoundError(f"Caption file not found: {caption_file_path}")

    # Read the caption from the text file
    with open(caption_file_path, "r") as f:
        caption = f.read().strip()

    return caption

# Step 3: Extract ingredients and presentation using LLM
def extract_details_with_llm(caption, llm):
    # Prompt to LLM for extracting details
    prompt = f"""
    Based on the caption: "{caption}", extract the following details:
    1. List all ingredients mentioned in the caption.
    2. Describe the method of presentation (e.g., sliced, plated, etc.).
    Provide the answer in JSON format with keys 'ingredients' and 'presentation'.
    """

    # Generate response
    response = llm(prompt, max_length=200)[0]["generated_text"]

    # Parse response into a dictionary
    try:
        details = json.loads(response)
    except json.JSONDecodeError:
        details = {"ingredients": "unknown", "presentation": "unknown"}

    return details

# Main Pipeline
def annotate_image(image_path, llm):
    # Step 1: Calculate brightness and saturation
    brightness, saturation = calculate_brightness_saturation(image_path)

    # Step 2: Read caption from the corresponding text file
    caption = read_caption_from_file(image_path)

    # Step 3: Extract details with LLM
    details = extract_details_with_llm(caption, llm)

    # Get filename
    file_name = os.path.basename(image_path)

    # Create annotation dictionary
    annotation = {
        "filename": file_name,
        "brightness": brightness,
        "color_saturation": saturation,
        "ingredients": details.get("ingredients", "unknown"),
        "presentation": details.get("presentation", "unknown"),
        "caption": caption,
    }

    return annotation

# Image path
image_path = "/content/Pizza_pugliese_0088.jpg"

# Run the pipeline
try:
    annotation = annotate_image(image_path, llm)

    # Print the annotation
    print(json.dumps(annotation, indent=4))

    # Save annotations to a JSON file (optional)
    with open("image_annotations.json", "w") as f:
        json.dump(annotation, f, indent=4)

    print("Annotations saved to image_annotations.json")
except FileNotFoundError as e:
    print(e)


{
    "filename": "Pizza_pugliese_0088.jpg",
    "brightness": 0.82,
    "color_saturation": 0.32,
    "ingredients": "unknown",
    "presentation": "unknown",
    "caption": "A pizza pugliese with bacon pieces and spinach leaves on top and cut into several pieces"
}
Annotations saved to image_annotations.json


In [13]:
# prompt: ambil informasi bahan pada file /content/Pizza_pugliese_0088.txt

import os
import json

# Image path
image_path = "/content/Pizza_pugliese_0088.jpg"

# Extract file name without extension
base_name = os.path.splitext(os.path.basename(image_path))[0]

# Construct the path for the text file
caption_file_path = os.path.join(os.path.dirname(image_path), f"{base_name}.txt")

# Check if the caption file exists
if not os.path.exists(caption_file_path):
    print(f"Caption file not found: {caption_file_path}")
else:
    # Read the caption from the text file
    with open(caption_file_path, "r") as f:
        caption = f.read().strip()

    # Now you have the caption in the 'caption' variable
    print(f"Caption from file: {caption}")

    #Further processing can be done here using the caption variable

Caption from file: A pizza pugliese with bacon pieces and spinach leaves on top and cut into several pieces


In [19]:
from transformers import pipeline
import re

# Menggunakan pipeline untuk ekstraksi teks
llm = pipeline("text2text-generation", model="google/flan-t5-large", framework="pt")

# Kalimat yang mengandung bahan makanan dan cara penyajian
caption = "A green bowl filled with es campur with pink sago and white shaved ice."

# Prompt untuk meminta model mengekstrak bahan makanan dan cara penyajian
prompt = f"""
Based on the following sentence, extract the ingredients and presentation (how it's served):
'{caption}'
"""

# Menggunakan model untuk ekstraksi bahan dan penyajian
response = llm(prompt, max_length=200)[0]["generated_text"]

# Menampilkan hasil
print("Extracted Response:", response)

# Ekstraksi bahan makanan menggunakan regex
ingredients = re.findall(r'\b\w+\b(?:\s+\w+)*\s(?:pieces|leaves|slices|chunks|dough|cheese|sauce)\b', caption)

# Ekstraksi cara penyajian menggunakan regex
presentation = re.findall(r'(cut into|served on|arranged|placed|on top|sliced|plated|in pieces|spread)', caption)

# Menampilkan hasil ekstraksi bahan dan penyajian
print("Extracted Ingredients:", ingredients)
print("Extracted Presentation:", presentation)

# Gabungkan hasil ekstraksi
annotation = {
    "ingredients": ingredients,
    "presentation": presentation
}

# Menampilkan hasil akhir
print("Final Annotation:", annotation)



Extracted Response: es campur, sago, white shaved ice
Extracted Ingredients: []
Extracted Presentation: []
Final Annotation: {'ingredients': [], 'presentation': []}


In [ ]:
cc3m = pd.read_csv("downloaded_cc3m_report.tsv.gz", compression="gzip", sep="\t", names=["caption", "path", "dataset", "mimetype", "size", "status", "url"])

In [ ]:
cc3m.iloc[0]

caption                            a very typical bus station
path        /export/home/.cache/lavis/conceptual_caption/i...
dataset                                                  cc3m
mimetype                                           image/jpeg
size                                                    36078
status                                                    200
url         http://lh6.ggpht.com/-IvRtNLNcG8o/TpFyrudaT6I/...
Name: 0, dtype: object

In [ ]:
len(cc3m)

3318333

In [ ]:
cnt = 0

valid_records = []

for i, path in tqdm(enumerate(cc3m.path.unique()), total=len(cc3m.path.unique())):
    path = str(path)
    if os.path.exists(path):
        record = cc3m.iloc[i]
        valid_records.append({"image": record["path"], "caption": record["caption"]})

        cnt += 1

print("Found {} valid records".format(cnt))

100%|██████████| 3130587/3130587 [17:28<00:00, 2986.08it/s]

Found 2759017 valid records


In [ ]:
len(valid_records)

2759017

In [ ]:
valid_records[1]

{'image': '/export/home/.cache/lavis/conceptual_caption/images/1_3239086386.jpg',
 'caption': 'sierra looked stunning in this top and this skirt while performing with person at their former university'}

In [ ]:
from omegaconf import OmegaConf


config_path = get_abs_path("configs/datasets/conceptual_caption/defaults_3m.yaml")

ann_path = OmegaConf.load(
    config_path
).datasets.conceptual_caption_3m.build_info.annotations.train.storage[0]

ann_path = get_cache_path(ann_path)

if os.path.exists(ann_path):
    # abort
    print("{} already exists".format(ann_path))
else:
    # Save the valid records to a json file
    with open(ann_path, "w") as f:
        f.write(json.dumps(valid_records))

/export/home/.cache/lavis/conceptual_caption/annotations/cc3m.json already exists


: 